![book](https://raw.githubusercontent.com/ageron/tensorflow-safari-course/master/images/intro_to_tf_course.png)

**Try not to peek at the solutions when you go through the exercises. ;-)**

First let's make sure this notebook works well in both Python 2 and Python 3:

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

In [2]:
import tensorflow as tf
tf.__version__

/home/farid/.virtualenvs/tensorflow-safari-course/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


'1.4.1'

*__From notebook 2 on variables:__*

In [ ]:
>>> graph = tf.Graph()
>>> with graph.as_default():
...     x = tf.Variable(100)
...     c = tf.constant(5)
...     increment_op = tf.assign(x, x + c)
...

## Collections

In [ ]:
>>> graph.get_collection(tf.GraphKeys.GLOBAL_VARIABLES)

In [3]:
tf.GraphKeys.GLOBAL_VARIABLES

'variables'

In [ ]:
>>> graph.add_to_collection("my_collection", c)
>>> graph.get_collection("my_collection")

## Navigating the Graph

In [ ]:
>>> graph = tf.Graph()
>>> with graph.as_default():
...     a = tf.constant(3)
...     b = tf.constant(5)
...     s = a + b
...
>>> graph.get_operations()

In [ ]:
>>> graph.get_operation_by_name("add") is s.op

In [ ]:
>>> graph.get_tensor_by_name("add:0") is s

In [ ]:
>>> list(s.op.inputs)

In [ ]:
>>> list(s.op.outputs)

## Naming Operations

In [ ]:
>>> graph = tf.Graph()
>>> with graph.as_default():
...     a = tf.constant(3, name='a')
...     b = tf.constant(5, name='b')
...     s = tf.add(a, b, name='s')
...


In [ ]:
>>> graph.get_operations()

## Exercise 3

![Exercise](https://c1.staticflickr.com/9/8101/8553474140_c50cf08708_b.jpg)

3.1) Create a graph with four variables named `"x1"`, `"x2"`, `"x3"` and `"x4"`, with initial values 1.0, 2.0, 3.0 and 4.0 respectively, then write some code that prints the name of every operation in the graph.

In [4]:
graph = tf.Graph()
with graph.as_default():
    x1 = tf.Variable(1.,name="x1")
    x2 = tf.Variable(2.,name="x2")
    x3 = tf.Variable(3.,name="x3")
    x4 = tf.Variable(4.,name="x4")


In [5]:
graph.get_operations()

[<tf.Operation 'x1/initial_value' type=Const>,
 <tf.Operation 'x1' type=VariableV2>,
 <tf.Operation 'x1/Assign' type=Assign>,
 <tf.Operation 'x1/read' type=Identity>,
 <tf.Operation 'x2/initial_value' type=Const>,
 <tf.Operation 'x2' type=VariableV2>,
 <tf.Operation 'x2/Assign' type=Assign>,
 <tf.Operation 'x2/read' type=Identity>,
 <tf.Operation 'x3/initial_value' type=Const>,
 <tf.Operation 'x3' type=VariableV2>,
 <tf.Operation 'x3/Assign' type=Assign>,
 <tf.Operation 'x3/read' type=Identity>,
 <tf.Operation 'x4/initial_value' type=Const>,
 <tf.Operation 'x4' type=VariableV2>,
 <tf.Operation 'x4/Assign' type=Assign>,
 <tf.Operation 'x4/read' type=Identity>]

3.2) Notice that for each `Variable`, TensorFlow actually created 4 operations:
* the variable itself,
* its initial value,
* an assignment operation to assign the initial value to the variable,
* and a read operation that you can safely ignore for now (for details, check out mrry's great answer to [this question](http://stackoverflow.com/questions/42783909/internals-of-variable-in-tensorflow)).

Get the collection of global variables in the graph, and for each one of them use `get_operation_by_name()` to find its corresponding `/Assign` operation (just append `"/Assign"` to the variable's name).

Hint: each object in the collection of global variables is actually a `Tensor`, not an `Operation` (it represents the variable's output, i.e., its value), so its name ends with `":0"`. You can get the `Operation` through the `Tensor`'s `op` attribute: its name will not end with `":0"`

In [16]:
[graph.get_operation_by_name(x.op.name+"/Assign") 
     for x  in graph.get_collection(tf.GraphKeys.GLOBAL_VARIABLES)]
    
    #

[<tf.Operation 'x1/Assign' type=Assign>,
 <tf.Operation 'x2/Assign' type=Assign>,
 <tf.Operation 'x3/Assign' type=Assign>,
 <tf.Operation 'x4/Assign' type=Assign>]

3.3) Add a `tf.group()` to your graph, containing all the assignment operations you got in question 3.2. Congratulations! You have just reimplemented `tf.global_variables_initializer()`.

Start a `Session()`, run your group operation, then evaluate each variable and print out the result.

3.4) For each assignment operation you fetched earlier, get its second input and store it in a list. Next, start a session and evaluate that list (using `sess.run()`). Print out the result: you should see `[1.0, 2.0, 3.0, 4.0]`. Can you guess why?

Try not to peek at the solution below before you have done the exercise! :)

![thinking](https://upload.wikimedia.org/wikipedia/commons/0/06/Filos_segundo_logo_%28flipped%29.jpg)

## Exercise 3 - Solution

3.1)

In [30]:
len(x1.op.inputs)

0

In [13]:
graph = tf.Graph()
with graph.as_default():
    x1 = tf.Variable(1.0, name="x1")
    x2 = tf.Variable(2.0, name="x2")
    x3 = tf.Variable(3.0, name="x3")
    x4 = tf.Variable(4.0, name="x4")

3.2)

In [14]:
gvars = graph.get_collection(tf.GraphKeys.GLOBAL_VARIABLES)
init_assign_ops = [graph.get_operation_by_name(gvar.op.name + "/Assign")
                   for gvar in gvars]

In [15]:
init_assign_ops

[<tf.Operation 'x1/Assign' type=Assign>,
 <tf.Operation 'x2/Assign' type=Assign>,
 <tf.Operation 'x3/Assign' type=Assign>,
 <tf.Operation 'x4/Assign' type=Assign>]

3.3)

In [ ]:
with graph.as_default():
    init = tf.group(*init_assign_ops)

In [ ]:
with tf.Session(graph=graph):
    init.run()
    print(x1.eval())
    print(x2.eval())
    print(x3.eval())
    print(x4.eval())

3.4)

In [20]:
init_assign_ops[0].inputs[1]

<tf.Tensor 'x1/initial_value:0' shape=() dtype=float32>

In [33]:
init_val_ops = [init_assign_op.inputs[1]
                for init_assign_op in init_assign_ops]

init_val_ops

[<tf.Tensor 'x1/initial_value:0' shape=() dtype=float32>,
 <tf.Tensor 'x2/initial_value:0' shape=() dtype=float32>,
 <tf.Tensor 'x3/initial_value:0' shape=() dtype=float32>,
 <tf.Tensor 'x4/initial_value:0' shape=() dtype=float32>]

In [31]:
with tf.Session(graph=graph) as sess:
    print(sess.run(init_val_ops))

[1.0, 2.0, 3.0, 4.0]


Explanation: in the case of assignment operations, the first input is a reference to the variable, and the second is the assignment value. The assignment operations we have here are used to initialize the variables, so their assignment values correspond to the initial values: 1.0 for `x1`, 2.0 for `x2`, 3.0 for `x3` and 4.0 for `x4`.